In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn 
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras
import pathlib

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, keras:
    print(module.__name__, module.__version__)

2.1.0
sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)
matplotlib 3.1.3
numpy 1.18.1
pandas 1.0.3
sklearn 0.22.1
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf


In [5]:
data_dir = '/home/kdd/python/fashion_image_label/'

In [6]:
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('/home/kdd/python/fashion_image_label')

In [7]:
for i in data_dir.glob('*'):
    print(i)

/home/kdd/python/fashion_image_label/fashion_test_jpg_10000.txt
/home/kdd/python/fashion_image_label/fashion_train_jpg_60000
/home/kdd/python/fashion_image_label/fashion_train_jpg_60000.txt
/home/kdd/python/fashion_image_label/fashion_test_jpg_10000


In [8]:
num_train_sample = len(list(data_dir.glob('fashion_train_jpg_60000/*')))
num_test_sample = len(list(data_dir.glob('fashion_test_jpg_10000/*')))

In [9]:
print(num_train_sample, num_test_sample)

60000 10000


In [10]:
for i in list(data_dir.glob('fashion_train_jpg_60000/*'))[:5]:
    print(i)
    print(tf.strings.split(str(i), os.path.sep)[-1].numpy())

/home/kdd/python/fashion_image_label/fashion_train_jpg_60000/43650_7.jpg
b'43650_7.jpg'
/home/kdd/python/fashion_image_label/fashion_train_jpg_60000/43050_6.jpg
b'43050_6.jpg'
/home/kdd/python/fashion_image_label/fashion_train_jpg_60000/41841_1.jpg
b'41841_1.jpg'
/home/kdd/python/fashion_image_label/fashion_train_jpg_60000/32955_4.jpg
b'32955_4.jpg'
/home/kdd/python/fashion_image_label/fashion_train_jpg_60000/32928_9.jpg
b'32928_9.jpg'


In [11]:
for i in data_dir.glob('*.txt'):
    print(i)

/home/kdd/python/fashion_image_label/fashion_test_jpg_10000.txt
/home/kdd/python/fashion_image_label/fashion_train_jpg_60000.txt


In [12]:
(data_dir/'fashion_train_jpg_60000.txt').open(encoding='utf-8').readlines()[:5]

['1_9.jpg 9\n', '2_0.jpg 0\n', '3_0.jpg 0\n', '4_3.jpg 3\n', '5_0.jpg 0\n']

In [13]:
label_info = (data_dir/'fashion_train_jpg_60000.txt').open(encoding='utf-8').readlines()

In [14]:
for i in label_info[:5]:
    print(i)

1_9.jpg 9

2_0.jpg 0

3_0.jpg 0

4_3.jpg 3

5_0.jpg 0



In [15]:
labels = [item.split()[-1] for item in label_info]

In [16]:
labels[:5]

['9', '0', '0', '3', '0']

In [32]:
def read_decode_img(img_path, label_info_dict):
    """读取解析jpg图片"""
#     img_path = img_path.numpy()

    print(img_path)
    label_key = img_path.split(os.path.sep)[-1]
#     print(label_key)
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img, label_info_dict[label_key]
    

In [18]:

tf.strings.split('/home/kdd/python/fashion_image_label/fashion_train_jpg_60000/43050_6.jpg', os.path.sep)[-1].numpy()

b'43050_6.jpg'

In [19]:
'/home/kdd/python/fashion_image_label/fashion_train_jpg_60000/43050_6.jpg'.split(os.path.sep)[-1]

'43050_6.jpg'

In [33]:
def make_datasets(data_dir, train=False):
    if train:
        images_path = str(data_dir/'fashion_train_jpg_60000/*')
        labels_path = data_dir/'fashion_train_jpg_60000.txt'
    else:
        images_path = str(data_dir/'fashion_test_jpg_10000/*')
        labels_path = data_dir/'fashion_test_jpg_10000.txt'
    label_info = (labels_path).open(encoding='utf-8').readlines()
    label_info_dict = {}
    for item in label_info:
        label_info_dict[item.split()[0]] = int(item.split()[-1])
#     labels = [int(item.split()[-1]) for item in label_info]
#     label_ds = tf.data.Dataset.from_tensor_slices(labels)
    
    image_path_ds = tf.data.Dataset.list_files(images_path)   
    image_label_ds = image_path_ds.map(lambda path: read_decode_img(img_path=path, label_info_dict=label_info_dict),
                                       num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
#     image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
    return image_label_ds

In [28]:
for i in tf.data.Dataset.list_files(str(data_dir/'fashion_train_jpg_60000/*')).batch(3).take(1):
    print(type(i))

<class 'tensorflow.python.framework.ops.EagerTensor'>


In [34]:
test_ds = make_datasets(data_dir).batch(2)

AttributeError: in converted code:

    <ipython-input-20-922b60c997b7>:16 None  *
        image_label_ds = image_path_ds.map(lambda path: read_decode_img(img_path=path, label_info_dict=label_info_dict),
    <ipython-input-32-e43bd25546e2>:4 read_decode_img  *
        tf.enable_eager_execution()

    AttributeError: module 'tensorflow' has no attribute 'enable_eager_execution'


In [ ]:
for i in test_ds.take(1):
    print(i)

In [ ]:
BATCH_SIZE = 32
train_ds = make_datasets(data_dir, train=True).repeat().shuffle(num_train_sample).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_ds = make_datasets(data_dir).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:

image_path_ds = tf.data.Dataset.list_files(str(data_dir/'fashion_train_jpg_60000/*')) 
for i in image_path_ds.take(5):
    print(i)

In [ ]:
model = keras.models.Sequential()
# 经过3次卷积(2层3*3的卷积核)池化(1层)操作
model.add(keras.layers.Conv2D(filters=32, kernel_size=5, padding='same',
                              activation='relu', input_shape=(28, 28, 1)))
model.add(keras.layers.Conv2D(filters=32, kernel_size=5, padding='same', activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
# model.add(keras.layers.BatchNormalization())
# model.add(keras.layers.Activation('selu'))

model.add(keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
# model.add(keras.layers.BatchNormalization())
# model.add(keras.layers.Activation('selu'))

model.add(keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
# model.add(keras.layers.BatchNormalization())
# model.add(keras.layers.Activation('selu'))
          
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.AlphaDropout(rate=0.5))

model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3)
]
history = model.fit(train_ds, epochs=10,
                    validation_data=test_ds,
                    steps_per_epoch=num_train_sample // BATCH_SIZE,
                    validation_steps = num_test_sample // BATCH_SIZE,
                    callbacks=callbacks)